<a href="https://colab.research.google.com/github/benchov/Machine_Learning_for_Trading_Knowledge/blob/main/Market_Analysis_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kneed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Market Analysis for Paper Trading





1.   Get Data
2.   Filter available assets in Alpaca Market
3.   Find Cointegrated pairs for Statistical Arbitrage
4.   Pick cointegrated pairs and detect trend with HMM Clustering
5.   Detect entry point, target price and stop price by the chart
6.   Create trading report.



#### Imports

In [10]:
from datetime import datetime
# remove unwanted warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# data extraction and management
import numpy as np
import pandas as pd
from pandas_datareader import DataReader
from pandas_datareader.nasdaq_trader  import get_nasdaq_symbols

# feature engineering
from sklearn.preprocessing import StandardScaler

# machine learing related imports
from sklearn.cluster import KMeans
from sklearn import metrics
from kneed import KneeLocator

# cointegration and statistic
from statsmodels.tsa.stattools import coint
import statsmodels.api as simplefilter

# reporting and visualisation
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

#### 1.0 Get Data

In [18]:
start_date = '2017-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')
symbols = get_nasdaq_symbols()
symbols = symbols[(symbols['ETF'] == True) & (symbols['Market Category'] == 'G')] # get global ETF's
symbols = list(symbols.index.values)
data = DataReader(name=symbols, start=start_date, end=end_date, data_source='yahoo')
data

/usr/local/lib/python3.8/dist-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'AADR', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/usr/local/lib/python3.8/dist-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'AAPB', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/usr/local/lib/python3.8/dist-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'AAPD', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/usr/local/lib/python3.8/dist-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'AAPU', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/usr/local/lib/python3.8/dist-packages/pandas_datareader/base.py:272: SymbolWarning: Failed to read symbol: 'AAXJ', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)
/usr/local/lib/python3.8/dist-packages/pandas_datareader/base.py:272: SymbolWarn

RemoteDataError: ignored

#### 1.1 Clear Data

In [ ]:
data.dropna(inplace=True, axis=1)
data.set_index('Date')
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
#### 2.0 Feature Engineering

In [ ]:
# create dataframe which cointains return and volatility infoemation
df_extended = pd.DataFrame(data.p)